In [1]:
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import base64
import csv
import os
import seaborn as sns
import numpy as np
from glob import glob
from shutil import copyfile

%matplotlib inline

pd.set_option('display.max_columns', 100)

In [2]:
PROJECT_DIR  = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences'
PICTURES_DIR = 'C:\\Users\\Natalia\\Documents\\GitHub\\Extracting-food-preferences\\classification_with_pics'

In [3]:
%cd $PROJECT_DIR

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences


In [4]:
#import modules
from utils import *
from vgg16 import Vgg16
from datetime import datetime

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

Using Theano backend.


In [5]:
%cd $PICTURES_DIR

#Set path to sample/ path if desired
path = PICTURES_DIR + '\\sample\\'
#path = PICTURES_DIR + '\\' + 'sample\\'
results_path=path + '\\results\\'
train_path=path + '\\train\\'
valid_path=path + '\\valid\\'

model_path = path + '\\models\\'
if not os.path.exists(model_path): os.mkdir(model_path)

C:\Users\Natalia\Documents\GitHub\Extracting-food-preferences\classification_with_pics


In [6]:
#Set constants
batch_size=25
no_of_epochs=1

In [18]:
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [19]:
batches.class_indices

{'popular': 0, 'unpopular': 1}

In [7]:
model = vgg_ft(2)

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [10]:
model.load_weights(results_path+'ft4.h5')

In [11]:
layers = model.layers

In [12]:
layers

In [13]:
last_conv_idx = [index for index,layer in enumerate(layers) 
                     if type(layer) is Convolution2D][-1]

In [14]:
last_conv_idx

30

In [15]:
layers[last_conv_idx]

In [16]:
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[last_conv_idx+1:]

In [17]:
fc_layers

In [20]:
start = datetime.now()

val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)

end = datetime.now()
total = end - start
print('Time to run the script on GPU is {}'.format(total))

In [21]:
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [22]:
start = datetime.now()

trn_features = conv_model.predict_generator(batches, batches.nb_sample)

end = datetime.now()
total = end - start
print('Time to run the script on GPU is {}'.format(total))

Time to run the script on GPU is 0:01:36.876519


In [24]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)

In [25]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [23]:
trn_features.shape

(200, 512, 14, 14)

In [26]:
# Such a finely tuned model needs to be updated very slowly!
opt = RMSprop(lr=0.001, rho=0.7)

In [30]:
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]
def get_fc_model():
    model = Sequential([
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(2, activation='softmax')
        ])

    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [31]:
fc_model = get_fc_model()

In [32]:
fc_model.fit(trn_features, trn_labels, nb_epoch=1, 
             batch_size=batch_size, validation_data=(val_features, val_labels))

Train on 200 samples, validate on 50 samples
Epoch 1/1
200/200 [==============================] - 11s - loss: 8.5791 - acc: 0.4650 - val_loss: 6.4472 - val_acc: 0.6000


In [33]:
fc_model.save_weights(model_path+'no_dropout.h5')